In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
369,tt1071812,25000000,20719451,Mao's Last Dancer,Chi Cao|Chengwu Guo|Wen Bin Huang|Joan Chen|Br...,Bruce Beresford,æœ€åŽçš„èˆžè€…,"At the age of 11, Li was plucked from a poor C...",117,Drama|Romance,Great Scott Productions Pty. Ltd.,10/1/2009,6.7,2009
645,tt0770752,70000000,111231041,Fool's Gold,Matthew McConaughey|Kate Hudson|Donald Sutherl...,Andy Tennant,This February True Love Takes a Dive.,Charming but luckless treasure hunter Ben Finn...,112,Comedy,De Line Pictures|Warner Bros.,2/7/2008,5.3,2008
1888,tt0163676,15000000,5217498,The In Crowd,Susan Ward|Lori Heuring|Matthew Settle|Nathan ...,Mary Lambert,What would you do to get in?,A mentally disturbed young woman takes a job a...,105,Thriller,Warner Bros. Pictures,7/19/2000,5.2,2000
1048,tt1433811,10000000,3428048,Disconnect,Jason Bateman|Hope Davis|Frank Grillo|Paula Pa...,Henry Alex Rubin,Look up.,Disconnect interweaves multiple storylines abo...,115,Drama|Thriller,Lionsgate|Liddell Entertainment|LD Entertainme...,9/11/2012,6.7,2012
744,tt1411238,25000000,147780440,No Strings Attached,Natalie Portman|Ashton Kutcher|Greta Gerwig|Ke...,Ivan Reitman,Friendship has its benefits.,Emma is a busy doctor who sets up a seemingly ...,108,Comedy|Romance,Paramount Pictures|Spyglass Entertainment|Kata...,1/12/2011,6.1,2011


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:
# вычисляем прибыль от фильма
data['profit'] = data['revenue'] - data['budget']

# вычисляем длину названия
data['original_title_len'] = list(map(lambda x: len(x), data['original_title']))

# вычисляем количество слов в описании
data['overview_words'] = list(map(lambda x: len(x.split(' ')), data['overview']))

# вычисляем месяц выпуска фильма
data['release_month'] = list(map(lambda x: x.split('/')[0],data['release_date']))


def get_unique_field_values(init_data, field_name):
    """возвращает уникальные значения заданного поля в котором значения разделены разделителем"""
    
    # собираем уникальные строки заданного поля 
    values_list = init_data[field_name].unique()
    # превращаем строки с значениями в списки
    values_splitted = map(lambda x: x.split('|'), values_list)

    # генерируем список уникальных значений
    unique_values = []
    # собираем списки воедино
    for value in values_splitted:
        unique_values = [*unique_values, *value]
    # выделяем уникальные значения списка
    unique_values = set(unique_values)
    
    return list(unique_values)


def get_genres_movies(init_data):
    """возвращает список жанров с количеством фильмов в них"""
    
    # возвращаем уникальный список жанров
    unique_genres = get_unique_field_values(init_data,'genres')

    # генерируем датафрейм с жанрами и количеством фильмов
    df_unique_genres = pd.DataFrame(list(map(lambda x: [x, 0], unique_genres)),columns=['Genre','Movies'])

    # подсчитываем количества фильмов в каждом жанре
    for index, genre in df_unique_genres.iterrows():      
        df_unique_genres.loc[df_unique_genres.Genre == genre['Genre'],'Movies'] \
            = init_data[init_data.genres.str.contains(genre['Genre'], na=False)].imdb_id.count()

    return df_unique_genres

def get_directors_movies(init_data):
    """возвращает список режисёров с количеством фильмов"""
    
    directors_movies = []
    
    # генерируем список уникальных режисёров
    directors_list = get_unique_field_values(init_data,'director')
    
    # перебираем режисёров и считаем ими снятые фильмы
    for director in directors_list:
        # возвращаем режисёром снятые фильмы
        movies = init_data[init_data.director.str.contains(director)]

        # инициализируем количество снятых фильмов
        movies_count = 0

        # перебираем снятые фильмы
        for index, movie in movies.iterrows():
            # возвращаем список режисёров фильма
            movie_dirs = movie.director.split('|')

            # отфильтровываем список на наличие анализируемого режисёра
            movie_dirs = list(filter(lambda x: x == director, movie_dirs))

            if len(movie_dirs):
                # режисёр присутствует в списке - увеличиваем количество снятых фильмов на единицу
                movies_count += 1

        # добавляем режисёра с количеством снятых фильмов в список
        directors_movies.append((director,movies_count))
    
    return directors_movies



#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [5]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
# answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = data.loc[data.budget == data.budget.max()].iloc[0].original_title
# если ответили верно, можете добавить комментарий со значком "+"

In [6]:
# тут пишем ваш код для решения данного вопроса:
# отфильтровываем данные где бюджет равен максимальному значению бюджета и берём первое значение
data.loc[data.budget == data.budget.max()].iloc[0].original_title

'Pirates of the Caribbean: On Stranger Tides'

ВАРИАНТ 2

In [7]:
# можно добавлять разные варианты решения
# сортируем данные по бюджету в убывающем порядке и берём первый элемент как ответ
sorted_data = data.sort_values(['budget'],ascending=False)
sorted_data.iloc[0].original_title

'Pirates of the Caribbean: On Stranger Tides'

# 2. Какой из фильмов самый длительный (в минутах)?

In [8]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
# отфильтровываем данные по продолжительности и выбираем строку с длиннейшей продолжительностью
answers['2'] = data.loc[data.runtime == data.runtime.max()].iloc[0].original_title

In [9]:
data.loc[data.runtime == data.runtime.max()].iloc[0].original_title

'Gods and Generals'

# 3. Какой из фильмов самый короткий (в минутах)?





In [10]:
# отфильтровываем данные по продолжительности и выбираем строку с наикротчайшей продолжительностью
answers['3'] = str(data.loc[data.runtime == data.runtime.min()].iloc[0].original_title)


In [11]:
data.loc[data.runtime == data.runtime.min()].iloc[0].original_title

'Winnie the Pooh'

# 4. Какова средняя длительность фильмов?


In [12]:
# вычисляем среднее значение продолжительности
answers['4'] = str(np.round(data.runtime.mean(),0))


# 5. Каково медианное значение длительности фильмов? 

In [13]:
# вычисляем медиану продолжительности фильмов
answers['5'] = str(np.round(data.runtime.median(),0))

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [14]:
# лучше код получения столбца profit вынести в Предобработку что в начале
# отфильтровываем данные по прибыли и выбираем строку с максимальным значением прибыли
answers['6'] = data[data.profit == data.profit.max()].iloc[0].original_title

# 7. Какой фильм самый убыточный? 

In [15]:
# отфильтровываем данные по прибыли и выбираем строку с минимальным значением прибыли
answers['7'] = data[data.profit == data.profit.min()].iloc[0].original_title

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [16]:
"""
отфильтровываем данные по прибыли и выбираем строки где прибыль больше нуля. 
В конце подсчитываем количество строк"""
answers['8'] = data[data.profit > 0].imdb_id.count()

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [17]:
# отбираем фильмы 2008 года
movies_2008 = data[data.release_year == 2008]

# отфильтровываем фильмы 2008 года по кассовым сборам и находим строку с максимальными сборами
answers['9'] = movies_2008[movies_2008.revenue == movies_2008.revenue.max()].iloc[0].original_title

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [18]:
# отбираем фильмы 2012 - 2014 годов
mv_2012_2014 = data[(data.release_year >= 2012) & (data.release_year <= 2014)]

# отфильтровываем фильмы 2012 - 2014 годов по прибыли и находим строку с минимальной прибылью
answers['10'] = mv_2012_2014[mv_2012_2014.profit == mv_2012_2014.profit.min()].iloc[0].original_title

# 11. Какого жанра фильмов больше всего?

In [19]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

# генерируем датафрейм с жанрами и количеством фильмов
df_unique_genres = get_genres_movies(data)

# записываем ответ    
answers['11'] = df_unique_genres.sort_values('Movies',ascending=False).iloc[0]['Genre']

ВАРИАНТ 2

In [20]:
# инициализируем список жанров с количеством фильмов в них
genres_movies_counts = []

# перебираем весь датафрейм с начальными данными
for index, movie in data.iterrows():
    # генерируем список жанров из поля с жанрами разделёнными |
    genres_list = movie.genres.split('|')
    
    # перебираем полученный список жанров фильма
    for gen in genres_list:
        # ищем жанр в списке жанров с количеством фильмов в них
        genre_idx = next((i for i, v in enumerate(genres_movies_counts) if v[0] == gen), None)
        
        # жанра в списке нет - добавляем его с колчеством фильмов равным 0
        if genre_idx == None:
            genres_movies_counts.append((gen,0))
            # опять считываем индекс жанра в списке жанров с количеством фильмов в них
            genre_idx = next((i for i, v in enumerate(genres_movies_counts) if v[0] == gen), None)
                
        """индекс жанра найден - прибавляем 1 к количеству фильмов в нём 
        путём создания нового объекта типа tuple 
        с названием жанра и увеличенным на единицу количеством фильмов"""
        genres_movies_counts[genre_idx] = (genres_movies_counts[genre_idx][0],genres_movies_counts[genre_idx][1] + 1)

# сортируем список жанров по количеству фильмов в них по убыванию
genres_movies_counts.sort(key=lambda x: x[1] * -1)        
# первый элемент списка является ответом на вопрос
answers['11'] = genres_movies_counts[0][0]
        

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [21]:
# выбираем прибыльные фильмы и считаем их колчество по жанрам
profitable_movies = get_genres_movies(data[data.profit > 0])

# сортируем список по колчиеству фильмов в убывающем порядке и выбираем первый элемент
answers['12'] = profitable_movies.sort_values('Movies',ascending=False).iloc[0]['Genre']

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [22]:
# собираем список уникальных режисёров
directors_list = get_unique_field_values(data,'director')

# инициализируем список режисёров с кассовыми сборами
directors_revenues = []

for director in directors_list:
    # подсчитываем кассовые сборы режисёра
    directors_revenues.append((director,data[data.director.str.contains(director)].revenue.sum()))
                                       
# сортируем список режисёров с кассовыми сборами по сборам в убывающем порядке    
directors_revenues.sort(key=lambda x: x[1] * -1)    

# записываем ответ
answers['13'] = directors_revenues[0][0]


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [23]:
# собираем фильмы в жанре Action
action_movies = data[data.genres.str.contains('Action')]

# собираем список режисёров, снимавших фильмы в жанре Action
directors_list = get_unique_field_values(action_movies,'director')

# инициализируем список режисёров со снятыми фильмами
directors_movies = get_directors_movies(action_movies)

# сортируем список по количеству фильмов    
directors_movies.sort(key=lambda x: x[1] * -1)

# первый элемент является ответом на вопрос
answers['14'] = directors_movies[0][0]

Вариант 2

In [70]:
# собираем фильмы в жанре Action
action_movies = data[data.genres.str.contains('Action')].copy()

# превращаем строку режисёров с разделителем на список
action_movies['director'] = action_movies.director.str.split('|')

# добавляем строки для списка режисёров в каждой строке изначальных данных
action_movies = action_movies.explode('director')

# создаём сводную таблицу по режисёрам и сборам
data_pivot = action_movies.pivot_table(index='director', values='imdb_id', aggfunc='count')

# сортируем сводную ьаблицу по количеству фильмов в убывающем порядке
data_pivot = data_pivot.sort_values(by='imdb_id', ascending=False)

# первый элемент и есть ответ
answers['14'] = data_pivot.iloc[0].name

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [60]:
# возвращаем список фильмов 2012 года
movies_2012 = data[data.release_year == 2012].copy()

# превращаем строку актёров с разделителем на список
movies_2012['cast'] = movies_2012.cast.str.split('|')

# добавляем строки для списка актёров в каждой строке изначальных данных
movies_2012 = movies_2012.explode('cast')

# создаём сводную таблицу по актёрам и сборам
data_pivot = movies_2012.pivot_table(index='cast', values='revenue', aggfunc='sum')

# сортируем сводную ьаблицу по сборам в убывающем порядке
data_pivot = data_pivot.sort_values(by='revenue', ascending=False)

# первый элемент и есть ответ
answers['15'] = data_pivot.iloc[0].name




Вариант 2

In [61]:
# возвращаем список фильмов 2012 года
movies_2012 = data[data.release_year == 2012].copy()

# собираем список актёров из отфильтрованного списка фильмов 2012 года
actors_list = get_unique_field_values(movies_2012,'cast')

# инициализация конечного списка с актёрами и сборами
actors_revenues = []

# перебираем список актёров
for actor in actors_list:    
    # подсчитываем кассовые сборы актёра в 2012 году
    actors_revenues.append((actor,data[(data.release_year == 2012) \
                                       & (data.cast.str.contains(actor))].revenue.sum()))
    
# сортируем список актёров с кассовыми сборами по сборам в убывающем порядке
actors_revenues.sort(key=lambda x: x[1] * -1)    

# записываем ответ
answers['15'] = actors_revenues[0][0]

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [25]:
# находим высокобюджетные фильмы
hb_movies = data[data.budget > data.budget.mean()]

# находим высокобюджетных фильмов актёров
actors_list = get_unique_field_values(hb_movies,'cast')

# инициализируем конечный список актёров с количеством фильмов
actors_movies = []

for actor in actors_list:
    # перебираем список актёров и добавляем актёров с количеством фильмов в конечный список
    actors_movies.append((actor,hb_movies[hb_movies.cast.str.contains(actor)].imdb_id.count()))
    
# сортируем конечный список по количеству фильмов    
actors_movies.sort(key=lambda x: x[1] * -1)    

# первый элемент списка является ответом на вопрос
answers['16'] = actors_movies[0][0]

Вариант 2

In [65]:
# находим высокобюджетные фильмы
hb_movies = data[data.budget > data.budget.mean()].copy()

# превращаем строку актёров с разделителем на список
hb_movies['cast'] = hb_movies.cast.str.split('|')

# добавляем строки для списка актёров в каждой строке изначальных данных
hb_movies = hb_movies.explode('cast')

# создаём сводную таблицу по актёрам и количеству фильмов
data_pivot = hb_movies.pivot_table(index='cast', values='imdb_id', aggfunc='count')

# сортируем сводную ьаблицу по количеству фильмов в убывающем порядке
data_pivot = data_pivot.sort_values(by='imdb_id', ascending=False)

answers['16'] = data_pivot.iloc[0].name

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [26]:
# выбираем фильмы с участием актёра Nicolas Cage
nc_movies = data[data.cast.str.contains('Nicolas Cage')]

"""
генерируем список жанров с количеством фильмов из списка фильмов с участием актёра Nicolas Cage.
сортируем полученный список и выбираем первый элемент как ответ
"""
answers['17'] = get_genres_movies(nc_movies).sort_values('Movies',ascending=False).iloc[0]['Genre']


# 18. Самый убыточный фильм от Paramount Pictures

In [27]:
# собираем фильмы от Paramount Pictures
pp_movies = data[data.production_companies.str.contains('Paramount Pictures')]

# записываем ответ (название фильма с минимальной прибылью)
answers['18'] = pp_movies[pp_movies['profit'] == pp_movies.profit.min()].original_title.iloc[0]


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [28]:
# вычисляем сумарные сборы по годам
revenues = data.groupby(['release_year']).sum()

# записываем ответ (ключ строки с наибольшими кассовыми сборами)
answers['19'] = revenues[revenues.revenue == revenues.revenue.max()].index[0]



# 20. Какой самый прибыльный год для студии Warner Bros?

In [29]:
# собираем фильмы от Warner Bros
wb_movies = data[data.production_companies.str.contains('Warner Bros')]

# группируем по годам и вычисляем суммы значений (сборы, прибыль и др.)
profits_by_years = wb_movies.groupby(['release_year']).sum()

# отфильтровываем сгруппированный список по прибыли и выбираем строку с максимальной прибылью
answers['20'] = profits_by_years[profits_by_years.profit == profits_by_years.profit.max()].index[0]


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [30]:
# группируем фильмы по месяцам и подсчитываем количество фильмов в каждом месяце
movies_by_months = data.groupby(['release_month']).count()

# отфильтровываем сгруппированные данные по количеству ид фильмов и выбираем строку с максимальным значением
answers['21'] = movies_by_months[movies_by_months.imdb_id == movies_by_months.imdb_id.max()].index[0]


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [31]:
# выбираем фильмы выпущеные в летние месяцы (с июня по август)
summer_movies = data[(data.release_month == '6') | (data.release_month == '7') | (data.release_month == '8')]

# подсчитываем количество ид фильмов
answers['22'] = summer_movies.imdb_id.count()


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [32]:
# выбираем в зимнее время выпущенные фильмы (с декабря по февраль)
winter_movies = data[(data.release_month == '12') | (data.release_month == '1') | (data.release_month == '2')]

# генерируем список режисёров с количеством снятых фильмов из зимой снятых фильмов
dir_winter_movies = get_directors_movies(winter_movies)

# сортируем сгенерированный список по количеству фильмов в убывающем порядке
dir_winter_movies.sort(key=lambda x: x[1] * -1)

# записываем ответ
answers['23'] = dir_winter_movies[0][0]




# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [33]:
# копируем начальные данные
data_24 = data.copy()

# превращаем строчный список компаний в список
data_24['production_companies'] = data_24.production_companies.str.split('|')

# добавляем в датафрейм с фильмами строки (в зависимости сколько строк в списке студий)
data_24 = data_24.explode('production_companies')

# групируем полученные данные по студиям и сортируем по убыванию.
data_24 = data_24.groupby(['production_companies'])['original_title_len'].mean().sort_values(ascending=False)

# записываем ответ
answers['24'] = data_24.index[0]

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [34]:
# копируем начальные данные
data_25 = data.copy()

# превращаем строчный список компаний в список
data_25['production_companies'] = data_25.production_companies.str.split('|')

# добавляем в датафрейм с фильмами строки (в зависимости сколько строк в списке студий)
data_25 = data_25.explode('production_companies')

# групируем полученные данные по студиям, вычисляем среднее значение и сортируем по убыванию.
data_25 = data_25.groupby(['production_companies'])['overview_words'].mean().sort_values(ascending=False)

# записываем ответ
answers['25'] = data_25.index[0]


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [35]:
# выбираем 1 процент фильмов отсортированных по рейтингу в убывающем порядке
one_percent_best = data.sort_values(['vote_average'],ascending=False)\
    .iloc[0:int(round(data.imdb_id.count() / 100,0))]

# выводим список фильмов и выбираем правильный ответ
one_percent_best.original_title

# записываем ответ
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [36]:
# крпируем данные в отдельный датафрейм
cast_pairs = data.copy()

# преобразуем значение поля в список
cast_pairs['cast'] = cast_pairs.cast.str.split('|')

# преобразуем значение поля из списка актёров в список пар актёров
cast_pairs.cast = cast_pairs.cast.apply(lambda x: list(combinations(sorted(x), 2))) 

# записываем часто повторяющиеся пары
cast_pairs.explode('cast').cast.mode()



full_pairs_list = []
# генерируем список всех пар
for cast_pair in cast_pairs['cast']:
    full_pairs_list = [*full_pairs_list, *cast_pair]
    
# убираем дубликаты    
full_pairs_list = list(set(full_pairs_list))

# инициализируем конечный список пар актёров
result_actors_pairs = []

# перебираем список пар актёров
for actors_pair in full_pairs_list:
    
    # проверяем снимались ли оба актёра в фильме из текущей строки
    if data[(data.cast.str.contains(str(actors_pair[0]))) \
            & (data.cast.str.contains(str(actors_pair[1])))].imdb_id.count():
        
        # актёры снимались - проверяем есть ли пара в конечном списке
        idx = next((i for i, v in enumerate(result_actors_pairs) if v[0] == actors_pair), None)

        if idx == None:
            # пара в конечном списке не найдена - добавляем
            result_actors_pairs.append((actors_pair,0))
            
            # опять ищем в конечном списке пару для нахождения индекса в конечном списке
            idx = next((i for i, v in enumerate(result_actors_pairs) if v[0] == actors_pair), None)
        
        # увеличиваем количество фильмов для пары актёров путём создания нового объекта tuple
        result_actors_pairs[idx] = (result_actors_pairs[idx][0], result_actors_pairs[idx][1] + 1)
                   
# сортируем конечный список по количеству фильмов в убывающем порядке                   
result_actors_pairs.sort(key=lambda x: x[1] * -1)                   

# просмотрев полученный список, записываем ответ
answers['27'] = 'Daniel Radcliffe & Rupert Grint'


ВАРИАНТ 2

# Submission

In [37]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides',
 '2': 'Gods and Generals',
 '3': 'Winnie the Pooh',
 '4': '110.0',
 '5': '107.0',
 '6': 'Avatar',
 '7': 'The Lone Ranger',
 '8': 1478,
 '9': 'The Dark Knight',
 '10': 'The Lone Ranger',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': 2015,
 '20': 2014,
 '21': '9',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [38]:
# и убедиться что ни чего не пропустил)
len(answers)

27